# main.ipynb – Datenextraktion & Vorverarbeitung
In diesem Notebook wird die strukturierte Verarbeitung von Online-Medienbeiträgen vorbereitet.  
Die wichtigsten Schritte:
- `.tar.gz`-Dateien entpacken (HTML-Dateien extrahieren)
- HTML-Inhalte bereinigen und Stoppwörter entfernen
- Wortfrequenzen je Medium und Datum zählen
- Ergebnisse in einer SQLite-Datenbank (`output/dwh.sqlite3`) speichern




In [36]:
# Benötigte Pakete importieren
# "Standard"-import
import os
import pandas as pd
# Einlesen & entpacken in html-Dateien
from glob import glob
import tarfile
# Kreation Mini-Data-Warehouse (SQL-Datenbank)
from datetime import datetime
import sqlite3
# Bearbeiten von html-Dateien
from bs4 import BeautifulSoup
import requests

In [37]:
# Pfade für das Projekt 

# Pfad für das Projekt
PROJECT_ROOT = os.getcwd()  

# Input, Rohdaten & html-Dateien
INPUT_PATH = os.path.join(PROJECT_ROOT, "input", "raw")
ZIP_PATH = os.path.join(INPUT_PATH, "downloaded_zips")
DATA_LAKE_PATH = os.path.join(INPUT_PATH, "data-lake")  

# Mini-Data-Warehouse bzw. SQL-Datenbank
LOGFILE_PATH = os.path.join(DATA_LAKE_PATH, "2025-02-08.csv")
log_file = pd.read_csv(LOGFILE_PATH)
SQL_PATH = os.path.join(PROJECT_ROOT, "output", "dwh.sqlite3")

# Stoppwörter einmalig laden
STOPWORDS_URL = "https://raw.githubusercontent.com/solariz/german_stopwords/master/german_stopwords_full.txt"
stopwords_list = requests.get(STOPWORDS_URL, allow_redirects=True).text.split("\n")[9:]



In [38]:
# Extrahieren & Einlesen einer .tar.gz-Datei
def extract_tar_file(ZIP_PATH, DATA_LAKE_PATH): 
    with tarfile.open(ZIP_PATH, "r:gz") as tar:
        for member in tar.getmembers():
            # Zielpfad für jede Datei bestimmen
            filename = os.path.basename(member.name)
            
            # Nur HTML-Dateien entpacken
            if filename.endswith(".html"):
                target_path = os.path.join(DATA_LAKE_PATH, filename)
                
                # Skippen, wenn Datei schon existiert
                if os.path.exists(target_path):
                    continue
                
                # Entpacken in HTML-Datei
                with open(target_path, "wb") as f:
                    f.write(tar.extractfile(member).read())
                
                print(f"Entpackt: {filename}")

In [39]:
# html-Datei mit Encoding einlesen
def read_html_file(filepath, encoding="utf-8"):
    # Datei öffnen und einlesen
    with open(filepath, "r", encoding=encoding) as f:
        return f.read()

# html-Inhalt bereinigen & in einzelne Wörter zerlegen
def process_html(html):
    # html-Parser von BeautifulSoup nutzen, um nur den Text auszulesen
    bstext = BeautifulSoup(html, "html.parser")
    text = bstext.get_text(separator=" ").lower()

    # Zeilenumbrüche durch Leerzeichen ersetzen & Wörter trennen
    items = text.replace("\n", " ").split(" ")

    # Kurze Wörter und Stoppwörter entfernen
    items = [i for i in items if len(i) > 1 and i not in stopwords_list]
    return items

In [40]:
# Einzelne Zeitungsdateien verarbeiten 
def process_newspaper(newspaper):
    filename = os.path.basename(newspaper["file_name"])  # Nur den Dateinamen rausholen
    full_path = os.path.join(DATA_LAKE_PATH, filename)   

    # Encoding aus der CSV einlesen
    encoding = newspaper["encoding"].lower()
    
    # html-Datei laden & bereinigen
    html = read_html_file(full_path, encoding)
    items = process_html(html)

   # Häufigkeit jedes Wortes zählen
    count = pd.Series(items).value_counts()

    # Als DataFrame formatieren und Zusatzinfos ergänzen
    count_df = count.to_frame()
    count_df.columns = ["count"]
    count_df["word"] = count_df.index
    count_df["paper"] = newspaper["name"]
    count_df["date"] = newspaper["date"]

    return count_df

In [41]:
# Liste für die Ergebnisse-Sammlung erstellen
collection = []

In [42]:
# Verarbeitung jeder einzelnen Datei und DataFrame mit Wortzählungen zurückgeben
def process_wrapper(newspaper):
    name = newspaper["name"]
    try:
        count = process_newspaper(newspaper)
        print(f"[INFO] Verarbeitung erfolgreich: {name}")
        collection.append(count)
    except Exception as e:
        print(f"[ERROR] Fehler bei {name}: {e}")

In [43]:
# Verarbeitung jeder Zeile
log_file.apply(process_wrapper, axis=1)

# Alle Ergebnisse in einer Tabelle sammeln
data = pd.concat(collection, axis=0)
print("Data shape:", data.shape)



[INFO] Verarbeitung erfolgreich: sz
[INFO] Verarbeitung erfolgreich: zeit
[INFO] Verarbeitung erfolgreich: faz
[INFO] Verarbeitung erfolgreich: heise
[INFO] Verarbeitung erfolgreich: golem
[INFO] Verarbeitung erfolgreich: tagesspiegel
[INFO] Verarbeitung erfolgreich: taz
[INFO] Verarbeitung erfolgreich: abendblatt
[INFO] Verarbeitung erfolgreich: berliner
[INFO] Verarbeitung erfolgreich: welt
[INFO] Verarbeitung erfolgreich: medium
[INFO] Verarbeitung erfolgreich: esslinger
[INFO] Verarbeitung erfolgreich: kdnuggets
[INFO] Verarbeitung erfolgreich: handelsblatt
[INFO] Verarbeitung erfolgreich: ntv
[INFO] Verarbeitung erfolgreich: pioneer
[INFO] Verarbeitung erfolgreich: suedwest
[INFO] Verarbeitung erfolgreich: t3n
[INFO] Verarbeitung erfolgreich: economist
[INFO] Verarbeitung erfolgreich: srf
[INFO] Verarbeitung erfolgreich: wef
[INFO] Verarbeitung erfolgreich: stuttgarter
[INFO] Verarbeitung erfolgreich: atlantic
[INFO] Verarbeitung erfolgreich: netzpolitik
[INFO] Verarbeitung erfolg

In [44]:
# In Datenbank speichern
# Erstellung SQL-Datenbank (dwh.sqlite3)
connection = sqlite3.connect(SQL_PATH)

# Wordcount-DataFrame in eine Tabelle erstellen
data.to_sql("wordcount", connection, index=False, if_exists="append")

# SQL-Datenbank schließen, um Speicherplatz freizugeben
connection.close()